In [3]:

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/app')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:

from audio.redis import *
from audio.audio import *
import io
from faster_whisper import WhisperModel

import json

import asyncio

model_size = "large-v3"
redis_client = await get_inner_redis()

model = WhisperModel(model_size, device="cuda", compute_type="float16")

ConnectionError: Error 113 connecting to redis:6379. 113.

In [ ]:
_,item = await redis_client.brpop('Audio2TranscribeQueue')
audio_name,client_id = item.split(':')
audio = Audio(audio_name,redis_client)

In [20]:
await audio.get()

True

In [21]:
segments, _ = model.transcribe(io.BytesIO(audio.data), beam_size=5,vad_filter=True,word_timestamps=True)
segments = [s for s in list(segments)]
transcription = [[w._asdict() for w in s.words] for s in segments]
await Transcript(audio_name,redis_client,transcription).save()
await redis_client.lpush(f'TranscribeReady:{audio_name}', 'Done')